In [1]:
import pyzx as zx
import random
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumCircuit, ParameterVector
import csv
import time
import japanize_matplotlib # 追加

In [2]:
def generate_H_S_CNOT_T_circuit(qubits, gates, p_t=0, seed=1000):
    random.seed(seed)  
    p_s = 0.333 * (1.0 - p_t)  
    p_had = 0.333 * (1.0 - p_t)  
    p_cnot = 0.333 * (1.0 - p_t)  

    c = zx.Circuit(qubits) 
    for _ in range(gates):
        r = random.random() 
        if r < p_had:
            c.add_gate("HAD", random.randrange(qubits))
        elif r < p_had + p_s:
            c.add_gate("S", random.randrange(qubits))
        elif r < p_had + p_s + p_t:
            c.add_gate("T", random.randrange(qubits))
        else:
            tgt = random.randrange(qubits)
            while True:
                ctrl = random.randrange(qubits)
                if ctrl != tgt:
                    break
            c.add_gate("CNOT", tgt, ctrl)
    return c

In [ ]:

c = generate_H_S_CNOT_T_circuit(4, 10000, p_t=0.1, seed=1000)

c = zx.optimize.basic_optimization(c)
g = c.to_graph()
zx.simplify.only_gadget(g)
#zx.full_reduce(g)
#c2 =  zx.extract.streaming_extract(g)
c2 =  zx.extract.extract_circuit(g, up_to_perm=True)

c2 = zx.optimize.basic_optimization(c2)





['Any', 'BaseGraph', 'Callable', 'Circuit', 'Dict', 'ET', 'EdgeType', 'FloatInt', 'Fraction', 'FractionLike', 'FrozenSet', 'Generic', 'Iterator', 'List', 'Literal', 'MatchBialgType', 'MatchCopyType', 'MatchGadgetType', 'MatchIdType', 'MatchLcompType', 'MatchObject', 'MatchPhasePolyType', 'MatchPivotType', 'MatchSpiderType', 'MatchSupplementarityType', 'MatchWType', 'Optional', 'Poly', 'RewriteOutputType', 'Set', 'Simplifier', 'Stats', 'Tuple', 'TypeVar', 'Union', 'VT', 'VertexType', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'apply_copy', 'apply_gadget_phasepoly', 'apply_rule', 'apply_supplementarity', 'bialg', 'bialg_iter', 'bialg_simp', 'cast', 'clifford_iter', 'clifford_simp', 'copy_simp', 'full_reduce', 'full_reduce_iter', 'gadget_iter', 'gadget_simp', 'get_w_io', 'get_w_partner', 'get_z_box_label', 'id_iter', 'id_simp', 'interior_clifford_iter', 'interior_clifford_simp', 'is_graph_like', 'itertools', 'jisaku

In [ ]:
# seed固定
seeds_list = [1000,1001,1002,1003,1004]

# パラメータ設定
qubits_list = [4, 8, 12, 16]
p_t_lists = [i * 0.04 for i in range(0, 11)]
gate_num_lists = [10000]
optimize_list = ["original","basic","clifford","gadget","full"]

# CSV書き込みの準備
output_file = "../kekka1220/1220ori_basic_cliffrod_10000.csv"
fieldnames = [
    "optimize_name","qubit","p_t","input_gates","seed","depth_bef","all_gates","one-qubit_gates","two-qubit gates","t","H","otherRZ","CNOT","CZ","depth_aft","time"
]
start_all = time.time()
with open(output_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for optimize_name in optimize_list:
        for qubit in qubits_list:
            for gate_num in gate_num_lists:
                for p_t in p_t_lists:
                    for seed in seeds_list:
                        try:
                            start = time.time()
                            # 回路作成
                            c = generate_H_S_CNOT_T_circuit(qubits = qubit,gates= gate_num,p_t = p_t,seed = seed)

                            depth_bef = QuantumCircuit.from_qasm_str(c.to_qasm()).depth()

                            if optimize_name == "basic":
                                c = zx.optimize.basic_optimization(c)
                            elif optimize_name == "clifford":
                                c = zx.optimize.basic_optimization(c)
                                g = c.to_graph()
                                zx.simplify.clifford_simp(g)
                                c = zx.extract.extract_circuit(g, up_to_perm=True)
                                c = zx.optimize.basic_optimization(c)
                            elif optimize_name == "original":
                                pass
                            elif optimize_name == "gadget":
                                c = zx.optimize.basic_optimization(c)
                                g = c.to_graph()
                                zx.simplify.only_gadget(g)
                                c = zx.extract.extract_circuit(g, up_to_perm=True)
                                c = zx.optimize.basic_optimization(c)
                            elif optimize_name == "full":
                                c = zx.optimize.basic_optimization(c)
                                g = c.to_graph()
                                zx.simplify.full_reduce(g)
                                c = zx.extract.extract_circuit(g, up_to_perm=True)
                                c = zx.optimize.basic_optimization(c)
                            else:
                                raise ValueError(f"Unknown optimization name: {optimize_name}")

                            end = time.time()

                            #qiskit変換とカウント
                            circuit_qiskit = QuantumCircuit.from_qasm_str(c.to_basic_gates().to_qasm())
                            depth_aft = circuit_qiskit.depth()
                            zx_dict = c.to_basic_gates().stats_dict()
                            qiskit_dict = circuit_qiskit.count_ops()

                            gates_aft = zx_dict["gates"]


                            # 結果をCSVに書き込む
                            writer.writerow({
                                "optimize_name": optimize_name,
                                "qubit": qubit,
                                "p_t": p_t,
                                "input_gates": gate_num,
                                "seed": seed,
                                "depth_bef": depth_bef,
                                "all_gates": gates_aft,
                                "depth_aft": depth_aft,
                                "t": zx_dict["tcount"],
                                "H": qiskit_dict.get("h", 0),
                                "otherRZ": gates_aft - qiskit_dict.get("h", 0) - qiskit_dict.get("cx", 0) - qiskit_dict.get("cz", 0) - zx_dict["tcount"],
                                "one-qubit_gates": gates_aft - qiskit_dict.get("cx", 0) - qiskit_dict.get("cz", 0),
                                "two-qubit gates": qiskit_dict.get("cx", 0) + qiskit_dict.get("cz", 0),
                                "CNOT": qiskit_dict.get("cx", 0),
                                "CZ": qiskit_dict.get("cz", 0),
                                "time": end - start
                            })

                            print(f"done. Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t},seed={seed},time={end-start}")

                        except Exception as e:
                            print(f"Error for Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t}: {e}")

end_all = time.time()
print(f"Total time: {end_all - start_all}")

done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1000,time=0.060312747955322266
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1001,time=0.09846854209899902
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1002,time=0.05454397201538086
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1003,time=0.05644822120666504
done. Qubits=4, Gate_Num=10000, P_T=0.0,seed=1004,time=0.0554661750793457
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1000,time=0.09543538093566895
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1001,time=0.0557708740234375
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1002,time=0.05480337142944336
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1003,time=0.05901360511779785
done. Qubits=4, Gate_Num=10000, P_T=0.04,seed=1004,time=0.09354472160339355
done. Qubits=4, Gate_Num=10000, P_T=0.08,seed=1000,time=0.05460858345031738
done. Qubits=4, Gate_Num=10000, P_T=0.08,seed=1001,time=0.05557751655578613
done. Qubits=4, Gate_Num=10000, P_T=0.08,seed=1002,time=0.05544161796569824
done. Qubits=4, Ga